In [1]:
#Dependancies
import pandas as pd
from sqlalchemy import *
from sqlalchemy.schema import CreateSchema
import pymysql
from config import username,pwd,host
import pymongo

In [2]:
#importing csv file
csvfile="data/Census_CSV_data.csv"
df=pd.read_csv(csvfile)

/anaconda3/envs/PythonEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Member ID: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Notes: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2016",1,1.0,3.51517e+07,...,...
1,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2011",2,2.0,3.34767e+07,...,...
2,2016,1,0,Canada,4.0,5.1,20000,1,"Population percentage change, 2011 to 2016",3,NaN,5,...,...
3,2016,1,0,Canada,4.0,5.1,20000,1,Total private dwellings,4,3.0,1.54124e+07,...,...
4,2016,1,0,Canada,4.0,5.1,20000,1,Private dwellings occupied by usual residents,5,4.0,1.40721e+07,...,...


In [4]:
#rows*columns
df.shape

(790944, 14)

In [5]:
#column names
df.columns

Index(['CENSUS_YEAR', 'GEO_CODE (POR)', 'GEO_LEVEL', 'GEO_NAME', 'GNR',
       'GNR_LF', 'DATA_QUALITY_FLAG', 'ALT_GEO_CODE',
       'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Member ID: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Notes: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Dim: Sex (3): Member ID: [1]: Total - Sex',
       'Dim: Sex (3): Member ID: [2]: Male',
       'Dim: Sex (3): Member ID: [3]: Female'],
      dtype='object')

In [6]:
#Selecting required columns
df=df[['CENSUS_YEAR','GEO_LEVEL','GEO_NAME','DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)','Dim: Sex (3): Member ID: [1]: Total - Sex','Dim: Sex (3): Member ID: [2]: Male','Dim: Sex (3): Member ID: [3]: Female']]
      

In [7]:
df.head()

,CENSUS_YEAR,GEO_LEVEL,GEO_NAME,DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
1,2016,0,Canada,"Population, 2011",3.34767e+07,...,...
2,2016,0,Canada,"Population percentage change, 2011 to 2016",5,...,...
3,2016,0,Canada,Total private dwellings,1.54124e+07,...,...
4,2016,0,Canada,Private dwellings occupied by usual residents,1.40721e+07,...,...


In [8]:
df.shape

(790944, 7)

In [9]:
# Renaming columns
df=df.rename(columns = {'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)': "Metric",'Dim: Sex (3): Member ID: [1]: Total - Sex':"Total",'Dim: Sex (3): Member ID: [2]: Male': 'Male','Dim: Sex (3): Member ID: [3]: Female': 'Female'})
df=df.rename(columns = {'CENSUS_YEAR':'Census_year','GEO_LEVEL':'Geo_level','GEO_NAME':'Geo_name'})
df.head()

,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
0,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
1,2016,0,Canada,"Population, 2011",3.34767e+07,...,...
2,2016,0,Canada,"Population percentage change, 2011 to 2016",5,...,...
3,2016,0,Canada,Total private dwellings,1.54124e+07,...,...
4,2016,0,Canada,Private dwellings occupied by usual residents,1.40721e+07,...,...


In [10]:
df.shape

(790944, 7)

In [11]:
df_population_2016=df.loc[df["Metric"]=="Population, 2016", :]
df_population_2016.insert(0,'Id',range(1,1+len(df_population_2016)))
df_population_2016.head()

,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
0,1,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
2247,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...
4494,3,2016,2,Avalon,"Population, 2016",86494,...,...
6741,4,2016,2,Bonavista--Burin--Trinity,"Population, 2016",74116,...,...
8988,5,2016,2,Coast of Bays--Central--Notre Dame,"Population, 2016",77680,...,...


In [12]:
df_population_2016.shape

(352, 8)

In [13]:
prov_2016=df_population_2016.loc[df_population_2016['Geo_level']==1,:]
prov_2016

,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
2247,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...
20223,10,2016,1,Prince Edward Island,"Population, 2016",142907,...,...
31458,15,2016,1,Nova Scotia,"Population, 2016",923598,...,...
58422,27,2016,1,New Brunswick,"Population, 2016",747101,...,...
83139,38,2016,1,Quebec,"Population, 2016",8.16436e+06,...,...
260652,117,2016,1,Ontario,"Population, 2016",1.34485e+07,...,...
534786,239,2016,1,Manitoba,"Population, 2016",1.27836e+06,...,...
568491,254,2016,1,Saskatchewan,"Population, 2016",1.09835e+06,...,...
602196,269,2016,1,Alberta,"Population, 2016",4.06718e+06,...,...
680841,304,2016,1,British Columbia,"Population, 2016",4.64806e+06,...,...


In [14]:
csvfile="data/Ridings.csv"
df_ridings=pd.read_csv(csvfile)
df_ridings

,Geo_name,Ridings
0,Newfoundland and Labrador,7
1,Prince Edward Island,4
2,Nova Scotia,11
3,New Brunswick,10
4,Ontario,121
5,Quebec,78
6,Saskatchewan,14
7,Manitoba,14
8,Alberta,34
9,British Columbia,42


In [15]:
prov_2016=prov_2016.merge(df_ridings,'left',on='Geo_name')
prov_2016



,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female,Ridings
0,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...,7
1,10,2016,1,Prince Edward Island,"Population, 2016",142907,...,...,4
2,15,2016,1,Nova Scotia,"Population, 2016",923598,...,...,11
3,27,2016,1,New Brunswick,"Population, 2016",747101,...,...,10
4,38,2016,1,Quebec,"Population, 2016",8.16436e+06,...,...,78
5,117,2016,1,Ontario,"Population, 2016",1.34485e+07,...,...,121
6,239,2016,1,Manitoba,"Population, 2016",1.27836e+06,...,...,14
7,254,2016,1,Saskatchewan,"Population, 2016",1.09835e+06,...,...,14
8,269,2016,1,Alberta,"Population, 2016",4.06718e+06,...,...,34
9,304,2016,1,British Columbia,"Population, 2016",4.64806e+06,...,...,42


In [16]:
# prov_2016.to_csv('data/prov_2016.csv')

In [17]:
csvfile="data/prov_2016.csv"
df_prov=pd.read_csv(csvfile)

df_prov

,Index,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female,Ridings,#ValueofVote,%ValueofVote
0,0,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...,7,1.367910,136.8
1,1,10,2016,1,Prince Edward Island,"Population, 2016",142907,...,...,4,2.869300,286.9
2,2,15,2016,1,Nova Scotia,"Population, 2016",923598,...,...,11,1.221840,122.2
3,3,27,2016,1,New Brunswick,"Population, 2016",747101,...,...,10,1.381580,138.2
4,4,38,2016,1,Quebec,"Population, 2016",8164361,...,...,78,0.993992,99.4
5,5,117,2016,1,Ontario,"Population, 2016",13448494,...,...,121,0.938692,93.9
6,6,239,2016,1,Manitoba,"Population, 2016",1278365,...,...,14,1.143110,114.3
7,7,254,2016,1,Saskatchewan,"Population, 2016",1098352,...,...,14,1.320740,132.1
8,8,269,2016,1,Alberta,"Population, 2016",4067175,...,...,34,0.867933,86.8
9,9,304,2016,1,British Columbia,"Population, 2016",4648055,...,...,42,0.938481,93.8


In [18]:
data = {
        "geo_name": df_prov['Geo_name'].values.tolist(),
        "population": df_prov['Total'].values.tolist(),
        "ridings": df_prov['Ridings'].values.tolist(),
        "val_vote":df_prov['%ValueofVote'].values.tolist(),
    }

data

{'geo_name': ['Newfoundland and Labrador',
  'Prince Edward Island',
  'Nova Scotia',
  'New Brunswick',
  'Quebec',
  'Ontario',
  'Manitoba',
  'Saskatchewan',
  'Alberta',
  'British Columbia',
  'Yukon',
  'Northwest Territories',
  'Nunavut'],
 'population': [519716,
  142907,
  923598,
  747101,
  8164361,
  13448494,
  1278365,
  1098352,
  4067175,
  4648055,
  35874,
  41786,
  35944],
 'ridings': [7, 4, 11, 10, 78, 121, 14, 14, 34, 42, 1, 1, 1],
 'val_vote': [136.8,
  286.9,
  122.2,
  138.2,
  99.4,
  93.9,
  114.3,
  132.1,
  86.8,
  93.8,
  287.9,
  247.6,
  287.3]}

In [19]:
prov_json="data/test_prov.json"
# prov=pd.read_json(prov_json)
prov_json_v2="data/test_prov_v2.json"

import json

with open(prov_json, 'r') as pj:
    data = json.load(pj)

features=data['features']
features

[{'type': 'Feature',
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[-123.3135247219999, 48.62434680600006],
      [-123.26837938599994, 48.60161920000007],
      [-123.29614031299991, 48.593931302000044],
      [-123.30641228499991, 48.599401698000065],
      [-123.3135247219999, 48.62434680600006]]],
    [[[-123.31669470599991, 48.731733605000045],
      [-123.29052821199991, 48.71499770000008],
      [-123.30568609099993, 48.70129400500008],
      [-123.33352158999992, 48.72012629100004],
      [-123.31669470599991, 48.731733605000045]]],
    [[[-123.2575850899999, 48.76752401500005],
      [-123.19039710699991, 48.75260388800007],
      [-123.18733580499992, 48.73414900900008],
      [-123.2075139079999, 48.73289998700005],
      [-123.24250998199994, 48.753899998000065],
      [-123.2575850899999, 48.76752401500005]]],
    [[[-123.1679149229999, 48.80750580200004],
      [-123.09930451299994, 48.78883730500007],
      [-123.08389370299994, 48.780941391000056],
      [

In [21]:
for feature in features:
    feature ['properties']['Population']=int((df_prov.loc[df_prov['Geo_name']==feature ['properties']['PRENAME'],'Total']).values[0])
    feature ['properties']['Val_vote']=float((df_prov.loc[df_prov['Geo_name']==feature ['properties']['PRENAME'],'#ValueofVote']).values[0])
    feature ['properties']['Ridings']=int((df_prov.loc[df_prov['Geo_name']==feature ['properties']['PRENAME'],'Ridings']).values[0])
features[1] ['properties']
# features[1] ['properties']['Val_vote']

{'PRUID': '24',
 'PRNAME': 'Quebec / Québec',
 'PRENAME': 'Quebec',
 'PRFNAME': 'Québec',
 'PREABBR': 'Que.',
 'PRFABBR': 'Qc',
 'Population': 8164361,
 'Val_vote': 0.993992,
 'Ridings': 78}

In [22]:
 with open(prov_json_v2, 'w') as pj2:
        json.dump(data, pj2)